In [1]:
import math
import numpy
import openmm.unit
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField, ParameterList
from openff.units import Quantity, unit
from smirnoff_plugins.utilities.openmm import simulate
from openmm.app import *
from openmm import *
from openmm.unit import *

In [2]:
from openff.toolkit.topology import Molecule
mol = Molecule.from_smiles("CCCC(O)CC(N)CCC", allow_undefined_stereo=True)
mol.generate_conformers(n_conformers=1)


In [3]:
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.interchange import Interchange
from openmm import unit as omm_unit

ff = ForceField("two_minima.offxml", load_plugins=True)
# ff = ForceField("openff-2.1.0.offxml", load_plugins=True)

ff.deregister_parameter_handler('ImproperTorsions')

top = mol.to_topology()
positions = mol.conformers[0]

interchange = Interchange.from_smirnoff(ff, topology=top, positions=positions)

system = interchange.to_openmm()


In [4]:
for force in system.getForces():
    print(force.__class__, force.getName())


<class 'openmm.openmm.NonbondedForce'> Nonbonded force
<class 'openmm.openmm.PeriodicTorsionForce'> PeriodicTorsionForce
<class 'openmm.openmm.HarmonicAngleForce'> HarmonicAngleForce
<class 'openmm.openmm.HarmonicBondForce'> HarmonicBondForce
<class 'openmm.openmm.CustomCompoundBondForce'> TwoMinima
<class 'openmm.openmm.CMMotionRemover'> CMMotionRemover


In [5]:
print(ff.registered_parameter_handlers)

['Constraints', 'Bonds', 'Angles', 'ProperTorsions', 'vdW', 'Electrostatics', 'LibraryCharges', 'ToolkitAM1BCC', 'TwoMinima']


In [6]:
# hh_force = [f for f in system.getForces() if f.getName() == "HarmonicHeight"][0]
# print(hh_force.getEnergyFunction())


In [7]:
bond_handler = ff.get_parameter_handler("Bonds")
angle_handler = ff.get_parameter_handler("Angles")
torsion_handler = ff.get_parameter_handler("ProperTorsions")
improper_handler = ff.get_parameter_handler("ImproperTorsions")
harmonic_handler = ff.get_parameter_handler("HarmonicHeights")
lee_krimm_handler = ff.get_parameter_handler("LeeKrimm")
two_minima_handler = ff.get_parameter_handler("TwoMinima")


print("Bond parameters:")
for param in bond_handler.parameters:
    print(param)

print("\nAngle parameters:")
for param in angle_handler.parameters:
    print(param)

print("\nTorsion parameters:")
for param in torsion_handler.parameters:
    print(param)

print("\nImproper torsion parameters:")
for param in improper_handler.parameters:
    print(param)

print("\nHarmonic height parameters:")
for param in harmonic_handler.parameters:
    print(param)

print("\nLee Krimm parameters:")
for param in lee_krimm_handler.parameters:
    print(param)

print("\nTwo Minima parameters:")
for param in two_minima_handler.parameters:
    print(param)

Bond parameters:
<BondType with smirks: [#6X4:1]-[#6X4:2]  id: b1  length: 1.527940216866 angstrom  k: 419.9869268191 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6X4:1]-[#6X3:2]  id: b2  length: 1.503434271105 angstrom  k: 484.1959214883 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6X4:1]-[#6X3:2]=[#8X1+0]  id: b3  length: 1.529478304416 angstrom  k: 418.6331368515 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6X3:1]-[#6X3:2]  id: b4  length: 1.466199291912 angstrom  k: 540.3345953498 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6X3:1]:[#6X3:2]  id: b5  length: 1.394445702699 angstrom  k: 765.1465671607 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6X3:1]=[#6X3:2]  id: b6  length: 1.382361687103 angstrom  k: 898.589948525 kilocalorie / angstrom ** 2 / mole  >
<BondType with smirks: [#6:1]-[#7:2]  id: b7  length: 1.46420197713 angstrom  k: 457.1029448115 kilocalorie / angstrom ** 2 / mole  >
<BondTy

In [8]:
from openmm import Context, Platform, VerletIntegrator
from openmm.app import Simulation
from openmm import unit as omm_unit
from openmm import Vec3


for i, f in enumerate(system.getForces()):
    f.setForceGroup(i)

integrator = VerletIntegrator(1.0 * omm_unit.femtoseconds)
platform = Platform.getPlatformByName("Reference")
context = Context(system, integrator, platform)

positions = mol.conformers[0].to_openmm()
context.setPositions(positions)

# positions = mol.conformers[0].to_openmm()
# positions[0] = positions[0] + Vec3(0.2, 0.0, 0.0) * omm_unit.nanometers
# context.setPositions(positions)


for i, f in enumerate(system.getForces()):
    state = context.getState(getEnergy=True, groups={i})
    print(f"Force {i} ({f.__class__.__name__}): {state.getPotentialEnergy()}")


Force 0 (NonbondedForce): -68.78270169789809 kJ/mol
Force 1 (PeriodicTorsionForce): 46.783618386402 kJ/mol
Force 2 (HarmonicAngleForce): 22.48495852616861 kJ/mol
Force 3 (HarmonicBondForce): 2.1910381765501334 kJ/mol
Force 4 (CustomCompoundBondForce): 23.580992396601438 kJ/mol
Force 5 (CMMotionRemover): 0.0 kJ/mol


In [9]:
for i, f in enumerate(system.getForces()):
    print(f"Force {i}: {f.__class__.__name__}, name: {f.getName()}")


Force 0: NonbondedForce, name: Nonbonded force
Force 1: PeriodicTorsionForce, name: PeriodicTorsionForce
Force 2: HarmonicAngleForce, name: HarmonicAngleForce
Force 3: HarmonicBondForce, name: HarmonicBondForce
Force 4: CustomCompoundBondForce, name: TwoMinima
Force 5: CMMotionRemover, name: CMMotionRemover
